In [ ]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
IMG_SIZE = (300, 300)
BATCH = 24

train_data_gener = ImageDataGenerator(rescale = 1./255, rotation_range = 20, 
                                width_shift_range = 0.1, height_shift_range = 0.1, 
                                horizontal_flip = True)
test_data_gener = ImageDataGenerator(rescale = 1./255)
train_data = train_data_gener.flow_from_directory('train', target_size = IMG_SIZE, 
                                                  batches = BATCH, class_mode = 'categorical')
test_data = test_data_gener.flow_from_directory('test', target_size = IMG_SIZE, 
                                                  batches = BATCH, class_mode = 'categorical')